In [1]:
!pip install -q flwr

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 598.1/598.1 kB 10.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.4/242.4 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.0/236.0 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.3/47.3 kB 2.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-api-core 1.34.1 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<4.0.0dev,>=3.19.5, but you have protobuf 4.25.8 which is incompatible.
google-spark-connect 0.5.2 requires google-api-core>=2.19.1, but you have google-api-core 1.34.1 which is incompatible.
google-cloud-bigtable 2.30.0 requires google-api-core[grpc]<3.0.0,>=2.16.0, but you have google-api-core 1.34.

In [2]:
!pip install -U flwr

In [18]:
# --- Imports ---
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import pandas as pd
import gc
import flwr as fl
from flwr.client import NumPyClient
from flwr.common import Context
from flwr.server import ServerApp, ServerConfig, ServerAppComponents
from flwr.server.strategy import FedProx
from flwr.simulation import run_simulation

# --- Load and prepare data ---
data = np.load("/kaggle/input/daicwoz-balanced/dataset_balanced.npz")
X_audio_test = data["X_audio_test"]
X_text_test = data["X_text_test"]
y_test = data["y_test"]
X_audio_dev = data["X_audio_dev"]
X_text_dev = data["X_text_dev"]
y_dev = data["y_dev"]
X_audio_train = data["X_audio_train"]
X_text_train = data["X_text_train"]
y_train = data["y_train"]

y_train = y_train.reshape(-1, 1)
y_dev = y_dev.reshape(-1, 1)
y_test = y_test.reshape(-1, 1)

# --- Parameters ---
NUM_CLIENTS = 10
input_shape_audio = (378, 60)
input_shape_text = (378, 9)

# --- Build model ---
def build_model():
    input_audio = layers.Input(shape=input_shape_audio, name='mfcc_input')
    x_audio = layers.LSTM(60, return_sequences=True, recurrent_dropout=0.002)(input_audio)
    x_audio = layers.BatchNormalization()(x_audio)
    x_audio = layers.Dropout(2e-4)(x_audio)
    x_audio = layers.LSTM(40, return_sequences=True, recurrent_dropout=0.002)(x_audio)
    x_audio = layers.BatchNormalization()(x_audio)
    x_audio = layers.Dropout(2e-4)(x_audio)
    x_audio = layers.LSTM(20, return_sequences=False, recurrent_dropout=0.002)(x_audio)
    x_audio = layers.BatchNormalization()(x_audio)
    x_audio = layers.Dropout(2e-4)(x_audio)

    input_text = layers.Input(shape=input_shape_text, name='text_input')
    x_text = layers.Flatten()(input_text)

    x = layers.Concatenate()([x_audio, x_text])
    x = layers.Reshape((1, -1))(x)
    x = layers.LSTM(20, return_sequences=False, recurrent_dropout=0.002)(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(2e-4)(x)
    x = layers.Flatten()(x)
    x = layers.Dense(15, activation='tanh')(x)
    x = layers.Dense(10, activation='tanh')(x)
    output = layers.Dense(1, activation='sigmoid')(x)

    return models.Model(inputs=[input_audio, input_text], outputs=output)

def rmse(y_true, y_pred):
    return tf.sqrt(tf.reduce_mean(tf.square(y_pred - y_true)))
rmse.__name__ = "rmse"

def compile_model():
    model = build_model()
    model.compile(optimizer=tf.keras.optimizers.RMSprop(1e-5),
                  loss=rmse,
                  metrics=['accuracy', 'mae', rmse])
    return model

# --- Split data across clients ---
def split_data(num_clients):
    size = len(y_train) // num_clients
    return [
        (X_audio_train[i * size : (i + 1) * size if i != num_clients - 1 else None],
         X_text_train[i * size : (i + 1) * size if i != num_clients - 1 else None],
         y_train[i * size : (i + 1) * size if i != num_clients - 1 else None])
        for i in range(num_clients)
    ]

client_datasets = split_data(NUM_CLIENTS)

# --- Client Definition ---
class DepressionClient(NumPyClient):
    def __init__(self, model, X_audio, X_text, y):
        self.model = model
        self.X_audio_train, self.X_audio_test, self.X_text_train, self.X_text_test, self.y_train, self.y_test = train_test_split(
            X_audio, X_text, y, test_size=0.1, random_state=42)

    def get_parameters(self, config=None):
        return self.model.get_weights()

    def set_parameters(self, parameters):
        self.model.set_weights(parameters)

    def fit(self, parameters, config):
        self.set_parameters(parameters)
        self.model.fit([self.X_audio_train, self.X_text_train], self.y_train,
                       batch_size=8, epochs=2, verbose=0, shuffle=True)
        return self.get_parameters(), len(self.y_train), {}

    def evaluate(self, parameters, config):
        self.set_parameters(parameters)
        loss, acc, mae, rmse_val = self.model.evaluate([self.X_audio_test, self.X_text_test], self.y_test, verbose=0)
        return float(loss), len(self.y_test), {
            "accuracy": float(acc), "mae": float(mae), "rmse": float(rmse_val)}

# --- Client App ---
def client_fn(context: Context):
    partition_id = int(context.node_config.get("partition-id", 0))
    model = compile_model()
    X_audio_c, X_text_c, y_c = client_datasets[partition_id]
    return DepressionClient(model, X_audio_c, X_text_c, y_c).to_client()

client_app = fl.client.ClientApp(client_fn=client_fn)

# --- Compare multiple FedProx configurations ---
mu_values = [0.01, 0.1, 1.0]  # 0.0 = FedAvg
results_by_mu = {}

for mu in mu_values:
    print(f"\n🚀 Starting FedProx with mu = {mu}")
    global_metrics = {"accuracy": [], "mae": [], "rmse": [], "f1_score": [], "precision": [], "recall": []}

    def server_fn(context: Context):
        model = compile_model()

        def evaluate_fn(server_round, parameters, _config):
            model.set_weights(parameters)
            y_pred = model.predict([X_audio_dev, X_text_dev])
            y_pred_labels = (y_pred > 0.5).astype(int)

            acc = accuracy_score(y_dev, y_pred_labels)
            f1 = f1_score(y_dev, y_pred_labels)
            prec = precision_score(y_dev, y_pred_labels)
            rec = recall_score(y_dev, y_pred_labels)
            mae_val = np.mean(np.abs(y_dev - y_pred))
            rmse_val = np.sqrt(np.mean((y_dev - y_pred) ** 2))

            global_metrics["accuracy"].append(acc)
            global_metrics["mae"].append(mae_val)
            global_metrics["rmse"].append(rmse_val)
            global_metrics["f1_score"].append(f1)
            global_metrics["precision"].append(prec)
            global_metrics["recall"].append(rec)

            print(f"📊 [μ={mu}] Round {server_round}: Acc={acc:.4f}, F1={f1:.4f}, RMSE={rmse_val:.4f}")
            tf.keras.backend.clear_session()
            gc.collect()

            return float(rmse_val), {
                "accuracy": float(acc),
                "f1_score": float(f1),
                "precision": float(prec),
                "recall": float(rec),
                "mae": float(mae_val),
                "rmse": float(rmse_val),
            }

        strategy = FedProx(
            fraction_fit=0.8,
            fraction_evaluate=0.5,
            min_fit_clients=NUM_CLIENTS,
            min_evaluate_clients=5,
            min_available_clients=NUM_CLIENTS,
            evaluate_fn=evaluate_fn,
            proximal_mu=mu,
        )
        return ServerAppComponents(strategy=strategy, config=ServerConfig(num_rounds=5))

    server_app = ServerApp(server_fn=server_fn)

    run_simulation(
        server_app=server_app,
        client_app=client_app,
        num_supernodes=NUM_CLIENTS,
        backend_config={"client_resources": {"num_cpus": 1, "num_gpus": 0.0}},
    )

    # --- Évaluation finale sur le jeu de test ---
    final_model = compile_model()
    final_model.set_weights(server_app._strategy.latest_parameters)

    y_pred_test = final_model.predict([X_audio_test, X_text_test])
    y_pred_test_labels = (y_pred_test > 0.5).astype(int)

    acc = accuracy_score(y_test, y_pred_test_labels)
    f1 = f1_score(y_test, y_pred_test_labels)
    prec = precision_score(y_test, y_pred_test_labels)
    rec = recall_score(y_test, y_pred_test_labels)
    mae = np.mean(np.abs(y_test - y_pred_test))
    rmse_val = np.sqrt(np.mean((y_test - y_pred_test) ** 2))

    results_by_mu[mu] = global_metrics
    results_by_mu[mu]['test_final'] = {
        "accuracy": acc,
        "f1_score": f1,
        "precision": prec,
        "recall": rec,
        "mae": mae,
        "rmse": rmse_val,
    }

    print(f"\n✅ [μ={mu}] FINAL TEST — Acc={acc:.4f}, F1={f1:.4f}, Prec={prec:.4f}, Rec={rec:.4f}, MAE={mae:.4f}, RMSE={rmse_val:.4f}")


🚀 Starting FedProx with mu = 0.01


INFO :      Starting Flower ServerApp, config: num_rounds=5, no round_timeout
INFO :      
INFO :      [INIT]
INFO :      Requesting initial parameters from one random client
(pid=34715) 2025-07-06 20:18:13.362561: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
(pid=34715) WARNING: All log messages before absl::InitializeLog() is called are written to STDERR
(pid=34715) E0000 00:00:1751833093.461387   34715 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
(pid=34711) E0000 00:00:1751833093.562928   34711 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
(ClientAppActor pid=34716) 2025-07-06 20:18:23.513151: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed

44/44 ━━━━━━━━━━━━━━━━━━━━ 15s 304ms/step
📊 [μ=0.01] Round 0: Acc=0.5330, F1=0.6409, RMSE=0.5009


INFO :      initial parameters (loss, other metrics): 0.5008765459060669, {'accuracy': 0.5330459770114943, 'f1_score': 0.6408839779005525, 'precision': 0.5206463195691203, 'recall': 0.8333333333333334, 'mae': 0.4997115731239319, 'rmse': 0.5008765459060669}
INFO :      
INFO :      [ROUND 1]
INFO :      configure_fit: strategy sampled 10 clients (out of 10)
(ClientAppActor pid=34715) 2025-07-06 20:18:48.536117: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)
(ClientAppActor pid=34714) 2025-07-06 20:18:48.600575: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)
INFO :      aggregate_fit: received 10 results and 0 failures


44/44 ━━━━━━━━━━━━━━━━━━━━ 6s 135ms/step
📊 [μ=0.01] Round 1: Acc=0.5158, F1=0.4608, RMSE=0.4977


...

44/44 ━━━━━━━━━━━━━━━━━━━━ 7s 141ms/step
📊 [μ=0.01] Round 4: Acc=0.5187, F1=0.4463, RMSE=0.5098


INFO :      fit progress: (4, 0.509807825088501, {'accuracy': 0.5186781609195402, 'f1_score': 0.4462809917355372, 'precision': 0.5252918287937743, 'recall': 0.3879310344827586, 'mae': 0.4899829924106598, 'rmse': 0.509807825088501}, 3822.183638161001)
INFO :      configure_evaluate: strategy sampled 5 clients (out of 10)
INFO :      aggregate_evaluate: received 5 results and 0 failures
INFO :      
INFO :      [ROUND 5]
INFO :      configure_fit: strategy sampled 10 clients (out of 10)
INFO :      aggregate_fit: received 10 results and 0 failures


44/44 ━━━━━━━━━━━━━━━━━━━━ 7s 138ms/step
📊 [μ=0.01] Round 5: Acc=0.5180, F1=0.4347, RMSE=0.5100


INFO :      fit progress: (5, 0.5100300908088684, {'accuracy': 0.5179597701149425, 'f1_score': 0.434709351305813, 'precision': 0.5254582484725051, 'recall': 0.3706896551724138, 'mae': 0.4911966919898987, 'rmse': 0.5100300908088684}, 4826.256102232001)
INFO :      configure_evaluate: strategy sampled 5 clients (out of 10)
INFO :      aggregate_evaluate: received 5 results and 0 failures
INFO :      
INFO :      [SUMMARY]
INFO :      Run finished 5 round(s) in 4841.74s
INFO :      	History (loss, distributed):
INFO :      		round 1: 0.5018168330192566
INFO :      		round 2: 0.5026693522930146
INFO :      		round 3: 0.5102518916130065
INFO :      		round 4: 0.4790216267108917
INFO :      		round 5: 0.5082826495170594
INFO :      	History (loss, centralized):
INFO :      		round 0: 0.5008765459060669
INFO :      		round 1: 0.49771133065223694
INFO :      		round 2: 0.49849215149879456
INFO :      		round 3: 0.5062850713729858
INFO :      		round 4: 0.509807825088501
INFO :      		round 5: 

AttributeError: 'FedProx' object has no attribute 'latest_parameters'

In [ ]:
# --- Résumé des résultats finaux sur le jeu de test ---
print("\n📊 Résultats finaux sur le test set :")
for mu in mu_values:
    res = results_by_mu[mu]["test_final"]
    print(f"μ={mu} — Acc={res['accuracy']:.4f}, F1={res['f1_score']:.4f}, RMSE={res['rmse']:.4f}")

# --- Tracer les résultats ---
metrics_to_plot = ["accuracy", "f1_score", "rmse"]
plt.figure(figsize=(15, 5))
for i, metric in enumerate(metrics_to_plot, 1):
    plt.subplot(1, len(metrics_to_plot), i)
    for mu in mu_values:
        plt.plot(results_by_mu[mu][metric], label=f"\u03bc={mu}")
    plt.title(metric.upper())
    plt.xlabel("Round")
    plt.ylabel(metric)
    plt.legend()
    plt.grid(True)

plt.tight_layout()
plt.suptitle("\ud83d\udcc8 Comparaison des performances selon \u03bc dans FedProx", fontsize=16, y=1.05)
plt.show()
